In [8]:
import tkinter as tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from RK4_models import rk4_sir, rk4_sir_demog, rk4_seir, rk4_seir_demog


class SIRModelApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.geometry("1200x800")
        self.root.title("SIR Model for Infectious Diseases")

        self.widget = None
        self.toolbar = None
        self.plot_label = None

        self._init_variables()
        self._init_frames()
        self._init_widgets()

        self.root.mainloop()

    def _init_variables(self):
        self.b = tk.DoubleVar()
        self.g = tk.DoubleVar()
        self.m = tk.DoubleVar()
        self.sig = tk.DoubleVar()
        self.days = tk.IntVar()
        self.sinitial = tk.DoubleVar()
        self.iinitial = tk.DoubleVar()
        self.einitial = tk.DoubleVar()
        self.statusmu = tk.IntVar()
        self.statuse = tk.IntVar()

    def _init_frames(self):
        self.left_frame = tk.Frame(self.root)
        self.right_frame = tk.Frame(self.root)
        self.button_frame = tk.Frame(self.root)

        self.left_frame.pack(side='left', fill='both', expand=True)
        self.right_frame.pack(side='left', fill='both', expand=True)
        self.button_frame.pack(side='bottom')

    def _init_widgets(self):
        self._create_slider(self.left_frame, 'β', self.b, 0, 2, 0.1, "Transmission coefficient")
        self._create_slider(self.left_frame, 'γ', self.g, 0, 1, 0.1, "Recovery rate")
        self._create_slider(self.left_frame, 'μ', self.m, 0, 0.5, 0.00001, "Birth/death rate", checkbutton=True, var=self.statusmu)
        self._create_slider(self.left_frame, 'σ', self.sig, 0, 1, 0.01, "Latent period")
        self._create_slider(self.left_frame, 'S₀', self.sinitial, 0, 1, 0.01, "Initial susceptible")
        self._create_slider(self.left_frame, 'I₀', self.iinitial, 0, 1, 0.01, "Initial infected")
        self._create_slider(self.left_frame, 'E₀', self.einitial, 0, 1, 0.01, "Initial exposed", checkbutton=True, var=self.statuse)
        self._create_slider(self.left_frame, 'Days', self.days, 1, 100, 1, "Simulation days", is_integer=True)

        self.calc_button = tk.Button(self.button_frame, text='Calculate', command=self.solve, height=2, width=10)
        self.quit_button = tk.Button(self.button_frame, text='Quit', command=self.root.destroy)
        self.calc_button.pack(side='left')
        self.quit_button.pack(side='left')

    def _create_slider(self, parent, label, variable, frm, to, res, info, checkbutton=False, var=None, is_integer=False):
        frame = tk.Frame(parent)
        info_button = tk.Button(frame, text='?', command=lambda: tk.messagebox.showinfo(f'{label} info', info))
        text_label = tk.Label(frame, text=label)
        slider = tk.Scale(frame, from_=frm, to=to, resolution=res, orient='horizontal', variable=variable)
        info_button.pack(side='left')
        text_label.pack(side='left')
        slider.pack(side='left')
        if checkbutton and var is not None:
            check = tk.Checkbutton(frame, text=f"Enable {label}", variable=var)
            check.pack(side='left')
        frame.pack(side='top')

    def solve(self):
    # Safely remove previous widgets if they exist
        if self.widget and self.widget.winfo_exists():
            self.widget.destroy()
            self.widget = None

        if self.toolbar and self.toolbar.winfo_exists():
            self.toolbar.destroy()
            self.toolbar = None

        if self.plot_label and self.plot_label.winfo_exists():
            self.plot_label.destroy()
            self.plot_label = None

        # Continue with solving and plotting...
        n = self.days.get()
        dt = 0.25 if n < 30 else 0.5 if n < 70 else 1

        s0 = self.sinitial.get()
        i0 = self.iinitial.get()
        e0 = self.einitial.get()
        beta = self.b.get()
        gamma = self.g.get()
        mu = self.m.get()
        sigma = self.sig.get()
        statusmu = self.statusmu.get()
        statuse = self.statuse.get()
        r0 = 1 - s0 - i0 - (e0 if statuse else 0)

        if r0 < 0:
            tk.messagebox.showerror("Input Error", "Initial population percentages exceed 100%.")
            return

        fig = Figure(figsize=(5, 5), dpi=100)
        ax = fig.add_subplot(111)
        t = list(range(n + 1))

        if statusmu and statuse:
            s, e, i, r = rk4_seir_demog(n, beta, gamma, mu, sigma, s0, e0, i0, r0, dt)
            ax.plot(t, s, 'r', label='Susceptible')
            ax.plot(t, e, 'k', label='Exposed')
            ax.plot(t, i, 'b', label='Infected')
            ax.plot(t, r, 'g', label='Recovered')
            label = 'Solved SEIR with Demography'
        elif not statusmu and statuse:
            s, e, i, r = rk4_seir(n, beta, gamma, sigma, s0, e0, i0, r0, dt)
            ax.plot(t, s, 'r', label='Susceptible')
            x.plot(t, e, 'k', label='Exposed')
            ax.plot(t, i, 'b', label='Infected')
            ax.plot(t, r, 'g', label='Recovered')
            label = 'Solved SEIR without Demography'
        elif statusmu and not statuse:
            s, i, r = rk4_sir_demog(n, beta, gamma, mu, s0, i0, r0, dt)
            ax.plot(t, s, 'r', label='Susceptible')
            ax.plot(t, i, 'b', label='Infected')
            ax.plot(t, r, 'g', label='Recovered')
            label = 'Solved SIR with Demography'
        else:
            s, i, r = rk4_sir(n, beta, gamma, s0, i0, r0, dt)
            ax.plot(t, s, 'r', label='Susceptible')
            ax.plot(t, i, 'b', label='Infected')
            ax.plot(t, r, 'g', label='Recovered')
            label = 'Solved SIR without Demography'

        ax.set_title(label)
        ax.legend()

        self.plot_label = tk.Label(self.right_frame, text=label)
        self.plot_label.pack()

        canvas = FigureCanvasTkAgg(fig, master=self.right_frame)
        self.toolbar = NavigationToolbar2Tk(canvas, self.right_frame)
        self.toolbar.update()
        self.widget = canvas.get_tk_widget()

        canvas.draw()
        self.widget.pack()


if __name__ == '__main__':
    app = SIRModelApp()

